<a href="https://colab.research.google.com/github/mwahaha-umich/ACLUFinalProject/blob/main/ACLU_Project_Part1_Combine_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this Jupyter notebook is to combine and split the data from the various source files we were provided. The current format is extremely redundant, will cause memory issues if handled directly in its entirety. By combining and splitting the data we will have a much easier time working with the dataset.

This notebook splits the Combined input into the following files:


*   dfMerged.csv - Includes all voting information (need to rename)
*   dfVoters.csv - Includes information with regard to the voters
*   dfOutreach.csv - Includes information based on which outreach effort(s) affected a particular voter.



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import requests

if 'google.colab' in str(get_ipython()):
  #We keep changing the different paths, so I'm going to check if various configurations exist.
  if (os.path.isdir("/content/drive/MyDrive/Shared with me/content/drive/My Drive/Shared with me/ACLU/")):
    GoogleDriveBase = "/content/drive/MyDrive/Shared with me/content/drive/My Drive/Shared with me/" #Anupriya
    WorkingDirectory = GoogleDriveBase + 'ACLU/' #Anupriya
  elif (os.path.isdir("/content/drive/MyDrive/Projects/ACLU")):
    GoogleDriveBase = "/content/drive/MyDrive/" #Mackenzie
    WorkingDirectory = GoogleDriveBase + "Projects/ACLU/" #Mackenzie
  else:  
    if (os.path.isdir("/content/drive/My Drive/Projects/ACLU/")):
      GoogleDriveBase = "/content/drive/My Drive/" #Kyle
      WorkingDirectory = GoogleDriveBase + "Projects/ACLU/" #Kyle
else: # We're not running in Google Colab, which means we're probably running locally. 
  #Put code here for local copies of the files
  GoogleDriveBase = "" 
  WorkingDirectory = GoogleDriveBase + "" 


WorkingFiles = WorkingDirectory + 'WorkingFiles/'
BasePickeDrive = GoogleDriveBase + WorkingDirectory + "Pickle/"


#Make the necessary folders for the script to run.
ListOfAllRequiredDirectories = [WorkingDirectory + 'Pickle', 
                          WorkingDirectory + 'WorkingFiles',
                          WorkingDirectory + 'AdditionalData',
                          WorkingDirectory + 'ACLUData']

for folder in ListOfAllRequiredDirectories:
  RunningPath = GoogleDriveBase + folder + "/"
  Path(RunningPath).mkdir(parents=True, exist_ok=True)

In [ ]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import csv
import string
import pandas as pd
import gc
import re
import string

In [ ]:
AllFilesInTheSubfolder = []
for subfolder in os.walk(WorkingDirectory + "ACLUData"):
  #Skip the pickle folder
  if "Pickle" in subfolder[0] or "WorkingFiles" in subfolder[0]:
      continue
  for file in subfolder[2]:
    FileWithPath = subfolder[0] +"/"+ file
    match = re.search("(.*)(text|mail|phone|call|postcards)", file.lower())
    if match == None:
      raise ValueError("Could not parse: " + file + "\n Please take a look at the type of communication and ensure that it's on the list. \nThis should only happen when we add new files whose name is inconsistant with the older files.")
    CommunicationType = match.group(2).replace("_", " ").strip()
    #We have some inconsistant communication type names. So, I'm doing a little cleanup here.
    if CommunicationType == "call":
      CommunicationType = "phone"
    if CommunicationType == "postcards":
      CommunicationType = "mail"
    FileWithDetails = (FileWithPath, match.group(1).replace("_", " ").strip(), match.group(2).replace("_", " ").strip(), file)
    AllFilesInTheSubfolder.append(FileWithDetails)
    #print(FileWithDetails)

In [ ]:
len(AllFilesInTheSubfolder)

25

In [ ]:
#Load all the files into dataframes
columns = []
dataframesbytype = {}

for file in AllFilesInTheSubfolder:
  df = pd.read_csv(file[0], sep='\t',header=(0))
  df["ElectionType"] = file[1]
  df["CommunicationType"] = file[2]
  df["File"] = file[3]
  columns.extend(df.columns)
  if file[1] not in dataframesbytype:
    dataframesbytype[file[1]] = []
  dataframesbytype[file[1]].append(df)
columns = set(columns)
print(len(dataframesbytype.keys()))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,28,58,59,60,61,62,65,69,71,73,75,77,79,82,83,85,87,89,91,93,95,96,97,98,99,100,101,114,115,116,117,118,119,120,121,122,123,125,126,127,129) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,36,80,84,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,36,76,78,80,84,90,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns

5


In [ ]:
print(dataframesbytype.keys())
for dfListKey in dataframesbytype.keys():
  print(len(dataframesbytype[dfListKey]))

dict_keys(['scomi', 'oakland prosecutor', 'oakland county prosecutor race', 'oakland', 'lpv'])
1
2
1
7
14


In [ ]:
len(columns)

137

In [ ]:
#Combine Dataframes of the same type
dataframesbytypejoined = {}
for dfListKey in dataframesbytype.keys():
  dataframesbytypejoined[dfListKey] = pd.concat(dataframesbytype[dfListKey])

In [ ]:
#Free up memory

del dataframesbytype
gc.collect()

50

In the following section we will be pulling out all unique voter information.

Please Note: As this data includes real world PII we will be making an effort to never show the data from this dataframe.

In [ ]:
#It looks like "Voter File VANID" is a unique key for each user, so I'm going to pull out any other person information and stick that into another dataframe. This will reduce memory footprint, which will be necessary for the pivot
#that's coming up.
dataframesbytypejoinedwithoutidvalues = {}

id_columns = ["Voter File VANID", "FirstName", "MiddleName", "LastName", "Address", "DWID", 'Zip5', 'Age', 'Primary19', 'MaritalStatus', 'mCity', 'Mass_Incarceration', 'StreetType', 'StreetSuffix',
  'mAddress', 'mState', 'mZip5', 'mZip4', 'Sex', 'City', 'State', 'Zip4', 'Suffix', 'CD', 'SD', 'HD', 'PreferredEmail', 'Preferred Phone', 'Cell Phone', 'CountyName', 'DOB', 'DateReg', 'Home Phone', 
  'EthnicCatalistName', 'Party', 'PersonalEmail', 'RaceName', 'StreetNo', 'StreetNoHalf', 'StreetPrefix', 'StreetName', 'AptType', 'AptNo', 'City.1', 'State.1', 'Zip5.1', 'Zip4.1', 
  '2020_Biden_Support', 'Voting_Aug_Prim', 'PoliceAccountability', 'VBM_Application', 'MarijuanaConviction', '2020_Biden_Support', 'Absentee_Voting'] #, 'VotedStatus', 'Voting_Nov_Gen'
primary_key = ["Voter File VANID"]
dfVoters = None
for dfListKey in dataframesbytypejoined.keys():
  df = dataframesbytypejoined[dfListKey]
  ColumnsToRemove = list(set(id_columns) - set(primary_key))
  if dfVoters is None:
    dfVoters = dataframesbytypejoined[dfListKey][id_columns]
    print(len(dfVoters))
    dataframesbytypejoinedwithoutidvalues[dfListKey] = df.drop(columns=ColumnsToRemove)
    continue
  print(len(dataframesbytypejoined[dfListKey]))
  dfVoters = dfVoters.append(dataframesbytypejoined[dfListKey][id_columns])
  dataframesbytypejoinedwithoutidvalues[dfListKey] = df.drop(columns=ColumnsToRemove)
  dataframesbytypejoined[dfListKey] = None
  gc.collect()
dfVoters = dfVoters.drop_duplicates()
dfVoters = dfVoters.sort_values(by=['FirstName', 'LastName'])

print(dfVoters)


149854
43512
142600
555506
2305359


In [ ]:
dfVoters.reset_index(inplace=True)
#dfVoters.to_csv(WorkingFiles + "dfVoters.csv", encoding='utf-8')

del dfVoters
gc.collect()

In [ ]:
del dataframesbytypejoined
gc.collect()

In [ ]:
dfMerged = pd.concat(dataframesbytypejoinedwithoutidvalues.values())

del dataframesbytypejoinedwithoutidvalues
gc.collect()

In [ ]:
#Pull out the file information, this indicates which outreach efforts they were included in.
dfOutreach = dfMerged[["Voter File VANID", "ElectionType", "CommunicationType", "File"]]
dfOutreach.reset_index(inplace=True)
#dfOutreach.to_csv(WorkingFiles + "dfOutreach.csv", encoding='utf-8')

del dfOutreach
gc.collect()

dfMerged = dfMerged.drop(columns=["ElectionType", "CommunicationType", "File"])

In [ ]:
print(dfMerged.columns)

In [ ]:

dfMerged.reset_index(inplace=True)


Finally, we will save the data to disk for use by the next Jupyter notebook.

[Merge Data](https://colab.research.google.com/drive/1epgGr9YFq1l3BMvDLApkWXnZ9TjCcJCo?usp=sharing)

In [ ]:

dfMerged.to_csv("/content/drive/MyDrive/content/drive/My Drive/Projects/ACLU/WorkingFiles/dfMerged.csv", encoding='utf-8')


In [ ]:
pip freeze > part1_requirements.txt